![snap](https://lever-client-logos.s3.amazonaws.com/2bd4cdf9-37f2-497f-9096-c2793296a75f-1568844229943.png)

# GetAround 

[GetAround](https://www.getaround.com/?wpsrc=Google+Organic+Search) is the Airbnb for cars. You can rent cars from any person for a few hours to a few days! Founded in 2009, this company has known rapid growth. In 2019, they count over 5 million users and about 20K available cars worldwide. 

As Jedha's partner, they offered this great challenges: 

## Context 

When renting a car, our users have to complete a checkin flow at the beginning of the rental and a checkout flow at the end of the rental in order to:

* Assess the state of the car and notify other parties of pre-existing damages or damages that occurred during the rental.
* Compare fuel levels.
* Measure how many kilometers were driven.

The checkin and checkout of our rentals can be done with three distinct flows:
* **📱 Mobile** rental agreement on native apps: driver and owner meet and both sign the rental agreement on the owner’s smartphone
* **Connect:** the driver doesn’t meet the owner and opens the car with his smartphone
* **📝 Paper** contract (negligible)

## Project 🚧

For this case study, we suggest that you put yourselves in our shoes, and run an analysis we made back in 2017 🔮 🪄

When using Getaround, drivers book cars for a specific time period, from an hour to a few days long. They are supposed to bring back the car on time, but it happens from time to time that drivers are late for the checkout.

Late returns at checkout can generate high friction for the next driver if the car was supposed to be rented again on the same day : Customer service often reports users unsatisfied because they had to wait for the car to come back from the previous rental or users that even had to cancel their rental because the car wasn’t returned on time.


## Goals 🎯

In order to mitigate those issues we’ve decided to implement a minimum delay between two rentals. A car won’t be displayed in the search results if the requested checkin or checkout times are too close from an already booked rental.

It solves the late checkout issue but also potentially hurts Getaround/owners revenues: we need to find the right trade off.

**Our Product Manager still needs to decide:**
* **threshold:** how long should the minimum delay be?
* **scope:** should we enable the feature for all cars?, only Connect cars?

In order to help them make the right decision, they are asking you for some data insights. Here are the first analyses they could think of, to kickstart the discussion. Don’t hesitate to perform additional analysis that you find relevant.

* Which share of our owner’s revenue would potentially be affected by the feature?
* How many rentals would be affected by the feature depending on the threshold and scope we choose?
* How often are drivers late for the next check-in? How does it impact the next driver?
* How many problematic cases will it solve depending on the chosen threshold and scope?

### Web dashboard

First build a dashboard that will help the product Management team with the above questions. You can use `streamlit` or any other technology that you see fit. 


### Machine Learning - `/predict` endpoint

In addition to the above question, the Data Science team is working on *pricing optimization*. They have gathered some data to suggest optimum prices for car owners using Machine Learning. 

You should provide at least **one endpoint** `/predict`. The full URL would look like something like this: `https://your-url.com/predict`.

This endpoint accepts **POST method** with JSON input data and it should return the predictions. We assume **inputs will be always well formatted**. It means you do not have to manage errors. We leave the error handling as a bonus.

Input example:

```
{
  "input": [[7.0, 0.27, 0.36, 20.7, 0.045, 45.0, 170.0, 1.001, 3.0, 0.45, 8.8], [7.0, 0.27, 0.36, 20.7, 0.045, 45.0, 170.0, 1.001, 3.0, 0.45, 8.8]]
}
```

The response should be a JSON with one key `prediction` corresponding to the prediction.

Response example:

```
{
  "prediction":[6,6]
}
```

### Documentation page

You need to provide the users with a **documentation** about your API.

It has to be located at the `/docs` of your website. If we take the URL example above, it should be located directly at `https://your-url.com/docs`).

This small documentation should at least include:
- An h1 title: the title is up to you.
- A description of every endpoints the user can call with the endpoint name, the HTTP method, the required input and the expected output (you can give example).

You are free to add other any other relevant informations and style your HTML as you wish.

### Online production

You have to **host your API online**. We recommend you to use [Heroku](https://www.heroku.com/) as it is free of charge. But you are free to choose any other hosting provider.

## Helpers 🦮

To help you start with this project we provide you with some pieces of advice:

* Spend some time understanding data 
* Don't overlook Data Analysis part, there is a lot of insights to find out. 
* Data Analysis should take 2 to 5 hours 
* Machine Learning should take 3 to 6 hours 
* You are not obligated to use libraries to handle your Machine Learning workflow like `mlflow` but we definitely advise you to do so.


### Share your code

In order to get evaluation, do not forget to share your code on a [Github](https://github.com/) repository. You can create a [`README.md`](https://guides.github.com/features/mastering-markdown/) file with a quick description about this project, how to setup locally and the online URL.

## Deliverable 📬

To complete this project, you should deliver:

- A **dashboard** in production (accessible via a web page for example)
- The **whole code** stored in a **Github repository**. You will include the repository's URL.
- An **documented online API** on Heroku server (or any other provider you choose) containing at least **one `/predict` endpoint** that respects the technical description above. We should be able to request the API endpoint `/predict` using `curl`:

```shell
$ curl -i -H "Content-Type: application/json" -X POST -d '{"input": [[7.0, 0.27, 0.36, 20.7, 0.045, 45.0, 170.0, 1.001, 3.0, 0.45, 8.8]]}' http://your-url/predict
```

Or Python:

```python
import requests

response = requests.post("https://your-url/predict", json={
    "input": [[7.0, 0.27, 0.36, 20.7, 0.045, 45.0, 170.0, 1.001, 3.0, 0.45, 8.8]]
})
print(response.json())
```

## Data 

There are two files you need to download: 

* [Delay Analysis](https://full-stack-assets.s3.eu-west-3.amazonaws.com/Deployment/get_around_delay_analysis.xlsx) 👈 Data Analysis 
* [Pricing Optimization](https://full-stack-assets.s3.eu-west-3.amazonaws.com/Deployment/get_around_pricing_project.csv) 👈 Machine Learning 


Happy coding! 👩‍💻

In [45]:
import requests
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from typing import List
from pydantic import BaseModel
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

import requests
import os
import joblib
from joblib import dump
import json

In [46]:
data_file_delay = 'get_around_delay_analysis.xlsx'
data_file_pricing = 'get_around_pricing_project.csv'
def load_data():
    data_delay = pd.read_excel(data_file_delay)
    data_pricing = pd.read_csv(data_file_pricing)
    return data_delay, data_pricing

In [47]:
df_delay, df_pricing = load_data()
print('df_delay')
display(df_delay.head())
print('df_pricing')
display(df_pricing.head())

df_delay


,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN


df_pricing


,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


In [48]:
# Première figure : scatter plot pour 'mileage' vs 'rental_price_per_day'
fig1 = px.scatter(df_pricing, x='mileage', y='rental_price_per_day', trendline='ols')

# Deuxième figure : scatter plot pour un 'engine_power' spécifique
power = 100
fig2 = px.scatter(df_pricing[df_pricing['engine_power'] == power],
                  x='mileage',
                  y='rental_price_per_day',
                  trendline='ols',
                  title=f'Rental price par mileage pour un engine power de {power}')

# Créer une figure avec des sous-graphiques
fig = make_subplots(rows=1, cols=2, subplot_titles=("Scatter Plot: Rental Price vs Mileage",
                                                    f"Scatter Plot: Rental Price for Engine Power {power}"))

# Ajouter les graphiques à la figure avec des sous-graphiques
fig.add_trace(fig1.data[0], row=1, col=1)  # Ajouter le premier scatter plot
fig.add_trace(fig1.data[1], row=1, col=1)  # Ajouter la ligne de tendance pour le premier plot

fig.add_trace(fig2.data[0], row=1, col=2)  # Ajouter le deuxième scatter plot
fig.add_trace(fig2.data[1], row=1, col=2)  # Ajouter la ligne de tendance pour le deuxième plot

# Mettre à jour les titres et la mise en page
fig.update_layout(title_text="Comparaison des prix de location en fonction du mileage et de la puissance pour 100",
                  showlegend=False)

# Afficher la figure combinée
fig.show()


In [49]:
# Supposons que df_delay soit déjà défini
# Première figure : scatter plot avec l'échelle y ajustée
fig1 = px.scatter(df_delay,
                  x='time_delta_with_previous_rental_in_minutes',
                  y='delay_at_checkout_in_minutes',
                  color='checkin_type')

# Calcul de la proportion pour le diagramme à secteurs
total = len(df_delay)
f_retards = len(df_delay[df_delay['delay_at_checkout_in_minutes'] > 0])
f_non_retards = len(df_delay[df_delay['delay_at_checkout_in_minutes'] <= 0])

# Création du DataFrame pour le pie chart
df_delay_pie = pd.DataFrame({
    'Type': ['Retards', 'A l\'heure'],
    'Proportion': [f_retards / total, f_non_retards / total]
})

# Deuxième figure : pie chart
fig2 = px.pie(df_delay_pie, values='Proportion', names='Type')

# Créer une figure avec des sous-graphiques et spécifier le type "domain" pour le pie chart
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=("Scatter Plot: Time Delta vs Delay", 
                                    "Proportion des Retards"),
                    specs=[[{'type': 'xy'}, {'type': 'domain'}]])

# Ajouter les graphiques à la figure principale
fig.add_trace(fig1.data[0], row=1, col=1)  # Ajouter le scatter plot
# Le scatter plot peut contenir plusieurs traces (par couleur), donc on les ajoute toutes
for trace in fig1.data[1:]:
    fig.add_trace(trace, row=1, col=1)

# Ajouter le pie chart dans la deuxième colonne
fig.add_trace(fig2.data[0], row=1, col=2)

# Ajuster manuellement l'échelle de l'axe y pour la première colonne (scatter plot)
fig.update_yaxes(range=[-4000, 11000], row=1, col=1)

# Mettre à jour la mise en page
fig.update_layout(title_text="Analyse des Retards et Proportions",
                  showlegend=True)

# Afficher la figure combinée
fig.show()


In [50]:
def calculate_proportions(df, checkin_type=None):
    if checkin_type:
        df = df[df['checkin_type'] == checkin_type]
    f_retards = len(df[df['delay_at_checkout_in_minutes'] > 0])
    f_non_retards = len(df[df['delay_at_checkout_in_minutes'] <= 0])
    return [f_non_retards, f_retards]

proportions_tout = calculate_proportions(df_delay)
proportions_mobile = calculate_proportions(df_delay, 'mobile')
proportions_connect = calculate_proportions(df_delay, 'connect')

fig = go.Figure()
fig.add_trace(go.Pie(labels=['À l\'heure', 'Retards'], values=proportions_tout, name="Mobile & Connect", sort=False))
fig.add_trace(go.Pie(labels=['À l\'heure', 'Retards'], values=proportions_mobile, name="Mobile", visible=False, sort=False))
fig.add_trace(go.Pie(labels=['À l\'heure', 'Retards'], values=proportions_connect, name="Connect", visible=False, sort=False))

# Mise à jour des tracés pour chaque bouton
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

# Ajout des boutons pour changer le diagramme
fig.update_layout(
    title_text="Proportion de retards",
    updatemenus=[{
        "buttons": [
            {
                "label": "Mobile & Connect",
                "method": "update",
                "args": [{"visible": [True, False, False]}]
            },
            {
                "label": "Mobile",
                "method": "update",
                "args": [{"visible": [False, True, False]}]
            },
            {
                "label": "Connect",
                "method": "update",
                "args": [{"visible": [False, False, True]}]
            }
        ],
        "direction": "down",
        "showactive": True,
    }]
)

fig.show()

In [51]:
fig = px.box(df_delay, y='delay_at_checkout_in_minutes', color='checkin_type', range_y=[-25000,72000])
fig.show()

In [52]:
# Proportion de retards supérieurs à chaque seuil
thresholds_in_minutes = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300]
scope = ['all', 'only_connect']

proportion_de_retards = np.zeros_like(thresholds_in_minutes)*1.0
df_delay2 = df_delay.copy()
mask = (df_delay2['delay_at_checkout_in_minutes'].notnull())
df_delay2 = df_delay2[mask]

for index in range(len(thresholds_in_minutes)):
    df_delay2[f"retard_trop_grand_{thresholds_in_minutes[index]}"] = df_delay2["delay_at_checkout_in_minutes"].apply(
                                                                        lambda x: 1.0 if x > thresholds_in_minutes[index] else 0.0)
    proportion_de_retards[index] = df_delay2[f"retard_trop_grand_{thresholds_in_minutes[index]}"].mean()

proportion_de_retards = pd.DataFrame(proportion_de_retards, columns=['prop_retards']).reset_index()
proportion_de_retards['seuils en minutes'] = proportion_de_retards['index'].apply(lambda i: thresholds_in_minutes[i])
proportion_de_retards = proportion_de_retards.drop(columns=['index'])  # Retirer la colonne 'index'
proportion_de_retards.head(11)


,prop_retards,seuils en minutes
0,0.575309,0
1,0.371834,30
2,0.268323,60
3,0.199254,90
4,0.156185,120
5,0.126575,150
6,0.107427,180
7,0.091888,210
8,0.081610,240
9,0.070843,270


In [53]:
proportion_de_retards_connect = np.zeros_like(thresholds_in_minutes)*1.0
df_delay3 = df_delay.copy()
mask = (df_delay3['delay_at_checkout_in_minutes'].notnull())
df_delay3 = df_delay3[mask]
mask2 = (df_delay3['checkin_type'] == 'connect')
df_delay3 = df_delay3[mask2]

for index in range(len(thresholds_in_minutes)):
    df_delay3[f"retard_trop_grand_{thresholds_in_minutes[index]}"] = df_delay3["delay_at_checkout_in_minutes"].apply(
                                                                        lambda x: 1.0 if x > thresholds_in_minutes[index] else 0.0)
    proportion_de_retards_connect[index] = df_delay3[f"retard_trop_grand_{thresholds_in_minutes[index]}"].mean()

proportion_de_retards_connect = pd.DataFrame(proportion_de_retards_connect, columns=['prop_retards']).reset_index()
proportion_de_retards_connect['index'] = proportion_de_retards_connect['index'].apply(lambda i: thresholds_in_minutes[i])

proportion_de_retards['prop_retards_connect'] = proportion_de_retards_connect['prop_retards']

In [54]:
proportion_de_retards_mobile = np.zeros_like(thresholds_in_minutes)*1.0
df_delay4 = df_delay.copy()
mask = (df_delay4['delay_at_checkout_in_minutes'].notnull())
df_delay4 = df_delay4[mask]
mask2 = (df_delay4['checkin_type'] == 'mobile')
df_delay4 = df_delay4[mask2]

for index in range(len(thresholds_in_minutes)):
    df_delay4[f"retard_trop_grand_{thresholds_in_minutes[index]}"] = df_delay4["delay_at_checkout_in_minutes"].apply(
                                                                        lambda x: 1.0 if x > thresholds_in_minutes[index] else 0.0)
    proportion_de_retards_mobile[index] = df_delay4[f"retard_trop_grand_{thresholds_in_minutes[index]}"].mean()

proportion_de_retards_mobile = pd.DataFrame(proportion_de_retards_mobile, columns=['prop_retards']).reset_index()
proportion_de_retards_mobile['index'] = proportion_de_retards_mobile['index'].apply(lambda i: thresholds_in_minutes[i])

proportion_de_retards['prop_retards_mobile'] = proportion_de_retards_mobile['prop_retards']

In [55]:
proportion_de_retards

,prop_retards,seuils en minutes,prop_retards_connect,prop_retards_mobile
0,0.575309,0,0.428865,0.613798
1,0.371834,30,0.258083,0.401731
2,0.268323,60,0.162551,0.296122
3,0.199254,90,0.111405,0.222342
4,0.156185,120,0.076720,0.177070
5,0.126575,150,0.056731,0.144932
6,0.107427,180,0.043210,0.124305
7,0.091888,210,0.031746,0.107695
8,0.081610,240,0.025573,0.096338
9,0.070843,270,0.018225,0.084672


In [56]:
thresholds_in_minutes = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300]
scope = ['all', 'only_connect']

mask = (df_delay['delay_at_checkout_in_minutes'].notnull())
mask_connect = (df_delay['checkin_type'] == 'connect')
mask_mobile = (df_delay[mask]['checkin_type'] == 'mobile')

proportion_de_retards = pd.DataFrame({
    'thresholds': thresholds_in_minutes,
    'prop_retards': [df_delay.loc[mask, 'delay_at_checkout_in_minutes'].gt(threshold).mean() for threshold in thresholds_in_minutes],
    'prop_retards_connect': [df_delay.loc[mask & mask_connect, 'delay_at_checkout_in_minutes'].gt(threshold).mean() for threshold in thresholds_in_minutes],
    'prop_retards_mobile': [df_delay.loc[mask & mask_mobile, 'delay_at_checkout_in_minutes'].gt(threshold).mean() for threshold in thresholds_in_minutes]
})

proportion_de_retards

,thresholds,prop_retards,prop_retards_connect,prop_retards_mobile
0,0,0.575309,0.428865,0.613798
1,30,0.371834,0.258083,0.401731
2,60,0.268323,0.162551,0.296122
3,90,0.199254,0.111405,0.222342
4,120,0.156185,0.076720,0.177070
5,150,0.126575,0.056731,0.144932
6,180,0.107427,0.043210,0.124305
7,210,0.091888,0.031746,0.107695
8,240,0.081610,0.025573,0.096338
9,270,0.070843,0.018225,0.084672


In [57]:
def calculate_proportions2(df):
    thresholds_in_minutes = [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300]
    mask = (df['delay_at_checkout_in_minutes'].notnull())
    mask_connect = (df['checkin_type'] == 'connect')
    mask_mobile = (df['checkin_type'] == 'mobile')
    proportions = pd.DataFrame({
        'thresholds': thresholds_in_minutes,
        'prop_retards': [df.loc[mask, 'delay_at_checkout_in_minutes'].
                        gt(threshold).mean() for threshold in thresholds_in_minutes],
        'prop_retards_connect': [df.loc[mask & mask_connect, 'delay_at_checkout_in_minutes'].
                                gt(threshold).mean() for threshold in thresholds_in_minutes],
        'prop_retards_mobile': [df.loc[mask & mask_mobile, 'delay_at_checkout_in_minutes'].
                                gt(threshold).mean() for threshold in thresholds_in_minutes]
    })
    return proportions

proportion_de_retards = calculate_proportions2(df_delay)

fig5 = go.Figure()
fig5.add_trace(
    go.Bar(
        x=proportion_de_retards["thresholds"],
        y=proportion_de_retards["prop_retards"],
        name="Proportion de retards"
    )
)
fig5.add_trace(
    go.Bar(
        x=proportion_de_retards["thresholds"],
        y=proportion_de_retards["prop_retards_mobile"],
        name="Proportion de retards (only mobile)"
    )
)
fig5.add_trace(
    go.Bar(
        x=proportion_de_retards["thresholds"],
        y=proportion_de_retards["prop_retards_connect"],
        name="Proportion de retards (only connect)"
    )
)
fig5.update_layout(
    title='Proportion de retards selon le seuil et le scope',
    xaxis_title='Seuil',
    yaxis_title='Proportion de retards',
    bargap=0.2
)
fig5.show()

In [58]:
def new_delay(row, threshold):
    if threshold > row['time_delta_with_previous_rental_in_minutes']:
        return row['delay_at_checkout_in_minutes'] - (threshold - row['time_delta_with_previous_rental_in_minutes'])
    else:
        return row['delay_at_checkout_in_minutes']

In [59]:
threshold = 210

# Crée un masque pour vérifier que les colonnes ne sont pas nulles
mask = (df_delay['delay_at_checkout_in_minutes'].notnull()
         & df_delay['time_delta_with_previous_rental_in_minutes'].notnull())

# Sélectionne les colonnes souhaitées et applique le masque directement sur le DataFrame original
df_delay2 = df_delay.loc[mask, ['delay_at_checkout_in_minutes', 'time_delta_with_previous_rental_in_minutes']].reset_index(drop=True)

# Ajoute les colonnes supplémentaires
df_delay2['is_delayed_0'] = (df_delay2['delay_at_checkout_in_minutes'] > 0)
df_delay2['new_delay_200'] = df_delay2.apply(lambda row: new_delay(row, threshold), axis=1)
df_delay2['is_delayed_200'] = df_delay2['new_delay_200'] > 0


In [60]:
df_delay2.head()

,delay_at_checkout_in_minutes,time_delta_with_previous_rental_in_minutes,is_delayed_0,new_delay_200,is_delayed_200
0,-15.0,570.0,False,-15.0,False
1,58.0,420.0,True,58.0,True
2,-76.0,330.0,False,-76.0,False
3,-6.0,630.0,False,-6.0,False
4,-7.0,90.0,False,-127.0,False


- Quelle part du revenu du propriétaire est potentiellement affectée par la fonctionnalité ?

Il est difficile de répondre à cette question avec les données actuelles. L'accès à des informations supplémentaires pourrait permettre une meilleure estimation des gains.

- Combien de locations sont concernées par la fonctionnalité selon la méthode de réservation et le seuil ?

Le premier graphique indique que les retards sont moins importants dans le cadre de la méthode "connect" par rapport à la méthode "mobile". Le quatrième graphique montre qu'en augmentant le seuil, le nombre de locations avec des retards diminue, ce qui pourrait affecter environ la moitié des locations.

- À quelle fréquence les conducteurs sont-ils en retard pour le prochain check-in ? Quel impact cela a-t-il sur le conducteur suivant ?

Selon le deuxième graphique, 57,5 % des retards sont observés. Le troisième graphique révèle que, pour le type "mobile", un nombre significatif de retards dépasse 720 minutes (12 heures), alors que seulement 13 retards sont enregistrés pour le type "connect". L'illustration montre que le retard impactera le client suivant si celui-ci dépasse le délai autorisé. Le premier graphique indique également que plus de locations sont affectées lorsque le délai est court (0 à 200 minutes), justifiant ainsi l'introduction d'un seuil.

- Combien de cas problématiques pourraient être résolus selon le seuil et le champ d'application choisis ?

Le quatrième graphique montre qu'en passant au type "connect", le taux de retards pourrait passer de 61 % à 43 %. En appliquant un seuil de 100 minutes, le nombre de retards pourrait être divisé par trois ou quatre. À mesure que le nombre de retards diminue, le nombre de cas problématiques diminue également.

# MACHINE LEARNING

In [61]:
X = df_pricing.drop('rental_price_per_day', axis=1)
y = df_pricing['rental_price_per_day']

categorical_features = ['model_key', 'fuel', 'paint_color', 'car_type',
                        'private_parking_available', 'has_gps', 'has_air_conditioning',
                        'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numeric_features = ['mileage', 'engine_power']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

regressor = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', LinearRegression())])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

regressor.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['mileage', 'engine_power']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['model_key', 'fuel',
                                                   'paint_color', 'car_type',
                                                   'private_parking_available',
                                                   'has_gps',
                                                   'has_air_conditioning',
                                                   'automatic_car',
                                                   'has_getaround_connect',
                                                   'has_speed_regulator',
                                                   'winter_tires'])])),
                ('regressor', LinearRegression())])

In [62]:
print("R2 score on training set : ", regressor.score(X_train, y_train))
print("R2 score on test set : ", regressor.score(X_val, y_val))

R2 score on training set :  0.7156703365824261
R2 score on test set :  0.6882520713344746


In [63]:
X = df_pricing.drop('rental_price_per_day', axis=1)
y = df_pricing['rental_price_per_day']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

categorical_features = ['model_key', 'fuel', 'paint_color', 'car_type']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

boolean_features = ['private_parking_available', 'has_gps', 'has_air_conditioning', 'automatic_car',
                    'has_getaround_connect', 'has_speed_regulator', 'winter_tires']
boolean_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first'))
])

numeric_features = ['mileage', 'engine_power']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('bool', boolean_transformer, boolean_features)
    ])

param_grid = {
    'regressor__n_estimators': [100, 500, 1000],
    'regressor__learning_rate': [0.01, 0.05, 0.1],
    'regressor__max_depth': [3, 4, 5],
}

xgbmodel = XGBRegressor(objective='reg:squarederror')

regressor = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', xgbmodel)])

gridsearch = GridSearchCV(estimator=regressor,
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',
                           cv=3,
                           verbose=1)

gridsearch.fit(X_train, y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['mileage',
                                                                          'engine_power']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['model_key',
                                                                          'fuel',
                                                                          'paint_color',
                                                                          'car_type']),
                                                                        ('bool',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(drop='f...
                                                     max_depth=None,
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=None, ...))]),
             param_grid={'regressor__learning_rate': [0.01, 0.05, 0.1],
                         'regressor__max_depth': [3, 4, 5],
                         'regressor__n_estimators': [100, 500, 1000]},
             scoring='neg_mean_squared_error', verbose=1)

In [64]:
y_train_pred = gridsearch.predict(X_train)
y_val_pred = gridsearch.predict(X_val)
print("Best parameters:", gridsearch.best_params_)
print("Best cross-validation score: {:.2f}".format(gridsearch.best_score_))

Best parameters: {'regressor__learning_rate': 0.05, 'regressor__max_depth': 4, 'regressor__n_estimators': 1000}
Best cross-validation score: -280.35


In [65]:
print("NMSE on training set : ", gridsearch.score(X_train, y_train))
print("NMSE on test set : ", gridsearch.score(X_val, y_val))

NMSE on training set :  -131.56601299018294
NMSE on test set :  -230.4999475874249


In [66]:
json_data = {
  "input": [4842,'Audi',195840,160,'diesel','grey','van',True,True,False,False,True,False,True]
}
feature_names = ['age', 'model_key', 'mileage', 'engine_power', 'fuel', 'paint_color', 'car_type',
                 'private_parking_available', 'has_gps', 'has_air_conditioning',
                 'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']
input_data = pd.DataFrame([json_data['input']], columns=feature_names)

predicted_price = gridsearch.predict(input_data)

output_json = {
  "prediction": predicted_price.tolist()
}

print(json.dumps(output_json))

{"prediction": [126.2121353149414]}


In [67]:
X = df_pricing.drop('rental_price_per_day', axis=1)
y = df_pricing['rental_price_per_day']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

categorical_features = ['model_key', 'fuel', 'paint_color', 'car_type']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

boolean_features = ['private_parking_available', 'has_gps', 'has_air_conditioning', 'automatic_car',
                    'has_getaround_connect', 'has_speed_regulator', 'winter_tires']
boolean_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first'))
])

numeric_features = ['mileage', 'engine_power']
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('bool', boolean_transformer, boolean_features)
    ])

xgbmodel = XGBRegressor(objective='reg:squarederror',
                        n_estimators = 500,
                        learning_rate = 0.1,
                        max_depth = 4)

regressor = Pipeline(steps=[('preprocessor', preprocessor),
                            ('regressor', xgbmodel)])

regressor.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['mileage', 'engine_power']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['model_key', 'fuel',
                                                   'paint_color', 'car_type']),
                                                 ('bool',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['private_parking_...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=4, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=500, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [68]:
print("R2 score on training set : ", regressor.score(X_train, y_train))
print("R2 score on test set : ", regressor.score(X_val, y_val))

R2 score on training set :  0.894054536415926
R2 score on test set :  0.7835719498862275


In [69]:
json_data = {
  "input": [4842,'Audi',195840,160,'diesel','grey','van',True,True,False,False,True,False,True]
}
feature_names = ['age', 'model_key', 'mileage', 'engine_power', 'fuel', 'paint_color', 'car_type',
                 'private_parking_available', 'has_gps', 'has_air_conditioning',
                 'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']
input_data = pd.DataFrame([json_data['input']], columns=feature_names)

predicted_price = regressor.predict(input_data)

output_json = {
  "prediction": predicted_price.tolist()
}

print(json.dumps(output_json))

{"prediction": [127.0735092163086]}


In [70]:
import os
from joblib import dump

print("Saving model...")
dump(regressor, 'pricing_model.joblib')
print(f"Model has been saved here: {os.getcwd()}")

Saving model...
Model has been saved here: c:\Users\mouto\Documents\Jedha\dsfsft28\PROJECT\TEST_07


In [71]:
class VehicleData(BaseModel):
    age: int
    model_key: str
    mileage: int
    engine_power: int
    fuel: str
    paint_color: str
    car_type: str
    private_parking_available: bool
    has_gps: bool
    has_air_conditioning: bool
    automatic_car: bool
    has_getaround_connect: bool
    has_speed_regulator: bool
    winter_tires: bool

class InputData(BaseModel):
    input: List[VehicleData]

data = {
    "input": [[4842,'Audi',195840,160,'diesel','grey','van',True,True,False,False,True,False,True],
            [4838,'Toyota',39743,110,'diesel','black','van',False,True,False,False,False,False,True]]
}

def predict(data: InputData):
    if 'input' not in data:
        raise ValueError("La clé 'input' est manquante dans les données fournies.")
    
    vehicle_dicts = [dict(zip(['age', 'model_key', 'mileage', 'engine_power',
                               'fuel', 'paint_color', 'car_type', 'private_parking_available',
                               'has_gps', 'has_air_conditioning', 'automatic_car',
                               'has_getaround_connect', 'has_speed_regulator', 'winter_tires'],
                               vehicle)) for vehicle in data['input']]
    
    input_data = pd.DataFrame(vehicle_dicts)

    pricing_model = joblib.load('pricing_model.joblib')

    prediction = pricing_model.predict(input_data)

    response = {
        "prediction": prediction.tolist()
    }

    return response

result = predict(data)
print(result)

{'prediction': [127.0735092163086, 126.91487884521484]}


In [ ]:
response = requests.post("http://localhost:4000/predict", json={
    "input": [
        {
            "age": 4842,
            "model_key": "Audi",
            "mileage": 195840,
            "engine_power": 160,
            "fuel": "diesel",
            "paint_color": "grey",
            "car_type": "van",
            "private_parking_available": True,
            "has_gps": True,
            "has_air_conditioning": False,
            "automatic_car": False,
            "has_getaround_connect": True,
            "has_speed_regulator": False,
            "winter_tires": True
        }
    ]
})
print(response.json())